# Gear Design - SpreadSheet 1 Calculator

In [62]:
#Gear Design
import numpy as np
import pandas as pd
import math
from math import sqrt, pi

#General
T = 300 #F
Rel = 99
FOS = 2
Bearing_Life = 120 #kh

In [12]:
#Input 
Pinput = 150
Winput = 2000

#Cooling Pump
Ppump = 2
Wpump = 3000 #rpms

#Electrical Gen
Pgen = 53
Wgen = 1800

#Actuators/Motors
Pmotor = {'max':68, 'min':0}
Wmotor = 667

#Layout in
X1 = 12
X2 = 8
Yt = 12
Z1 = 4
Z2 = 8

i=1

#Test Ng
Ng = 60

In [93]:
eIC_motor = Winput/Wmotor
eIC_pump = Winput/Wpump
eIC_gen = Winput/Wgen

print("eIC+motor:", eIC_motor)
print("eIC+pump:",eIC_pump)
print("eIC+gen:",eIC_gen, "\n")

Np_eIC_pump = Ng/eIC_pump
Np_eIC_gen = Ng/eIC_gen
Np_eIC_motor = Ng/eIC_motor

print("Np(eIC+pump):",Np_eIC_pump)
print("Np(eIC+gen):",Np_eIC_gen)
print("Np(eIC+motor):",Np_eIC_motor, "\n")

#C = dp+dg/2
#e = dg/dp
#Ni = di*P

A = np.matrix([[0.5,0.5,0,0,0,0], [eIC_motor,1,0,0,0,0], [0,0,0.5,0.5,0,0],
        [0,0,-eIC_pump,1,0,0],[0,0,0,0,0.5,0.5],[0,0,0,0,-eIC_gen,1]])
# print(A, "\n")


B = np.matrix([[X1],[0],[sqrt(X2**2+((Yt/2)**2))],[0],[sqrt(X2**2+((Yt/2)**2))],[0],])
# print(B, "\n")

x = np.matmul(np.linalg.inv(A), B)

x = pd.DataFrame(x,index=(['dp_motor','dg_motor','dp_pump','dg_pump','dp_gen','dg_gen']))

Xmatrix = x[0]
print(Xmatrix, "\n")


P_pump = Np_eIC_pump/Xmatrix['dp_pump']
P_motor = Np_eIC_motor/Xmatrix['dp_motor']
P_gen = Np_eIC_gen/Xmatrix['dp_gen']

print("Pump:", P_pump)
print("Motor:",P_motor)
print("Gen:",P_gen)



eIC+motor: 2.998500749625187
eIC+pump: 0.6666666666666666
eIC+gen: 1.1111111111111112 

Np(eIC+pump): 90.0
Np(eIC+gen): 54.0
Np(eIC+motor): 20.01 

dp_motor   -12.009002
dg_motor    36.009002
dp_pump     12.000000
dg_pump      8.000000
dp_gen       9.473684
dg_gen      10.526316
Name: 0, dtype: float64 

Pump: 7.499999999999999
Motor: -1.6662499999999998
Gen: 5.7


# FOS Calculator from class 4/25 (incomplete)

### Planning on making this an interactive GUI for multiple sections (motor, pump, gen ect)

17 tooth 20 degree pressure angle spur pinion rotates @ 1800 rev/min & transmits 4 HP to a 52 tooth spur gear. 
Cuntual pitch is 10 teet/in the force width is 1.5 in & qaulity standard is No. 6.
The gears are standard mounted /w bearings immeadialty adjacent the pinion is a grade I steel w/a hardness of 240 brinell tooth surface through hardened core. The gear is steel through harbored grade I material w/a brinell hardness of 200 tooth surface and core.Poisssions ratio is 0.30,3p=0.30,Jg=0.40 Youngs modulus is 30 MPa. The leading is smooth becuase of motor & load.
Asume a pinion life of 10^8 cycles & reliability of 0.90 Yn= 1.3558 N^-0.0178, Zn=1.4488N^-0.023

In [5]:
#Book example in class 4/25

# Sf_p = St*Yn/(Kt*Kr)/sigma #5.62

# St(endurance strength) -->fig 4-2 Grade 1 Steel St= 77.3 Hb+ 12800 psi St_p = 31.350 psi St_g = 28260 psi
# Yn(load cycle factor) fig 14-14
#Yn_p = 0.977
#Yn_g = 1.3558(10**8/(52/17))**-0.0178 = 0.966

from math import pi,sqrt
P_life = 10**8

Yn_p = 1.3558*P_life**-0.0178
Yn_g = 1.3558*((10**8/(52/17))**-0.0178)

print('Ynp:',round(Yn_p,3),'Yng:',round(Yn_g,3))

Kt =1 #T<250 F, Kt=1
Kr = 0.88 # table value
Np = 17
Ng=52
p = 10
np = 1800
dp= Np/p
V = (pi*dp*np)/12
H =4 #hp
Mg =Np/Ng

Wt = 33000*H/V
print('Wt:',round(Wt,3))

K_o = 1

Qv=6
B = 0.8255
A = 59.77
K_v = (A+sqrt(V)/A)**B
Fw =1.5
Yp = 0.303
#Yg = interp 50 0.409 ->52 x -> 60 0.422
Yg = 0.412 # from interp
K_sp = 1.192*(Fw*sqrt(Yp)/p)**0.0535
K_sg = 1.192*(Fw*sqrt(Yg)/p)**0.0535


print('Ksp:',round(K_sp,3),'Ksg:',round(K_sg,3))

Cmc = 1
Cpf = Fw/10*dp- 0.0375 + 0.0125*Fw #0.0695
Cpm = 1
Af =0.127
Bf = 0.0158
Cf = -0.930*10**-4
Cma = (Af+Bf*Fw+Cf*Fw**2) #eqn 14-34 0.15
Ce = 1 #No adjustments
Km = 1+Cmc*(Cpf*Cpm+Cma*Ce) #1.22
print ('Km:',round(Km,3))
Kb = 1 #solid
#sigma_p = Wt*K_o*K_v*K_sp*Pd/Fw*(Km*Kb/J) #6417 psi
#sigma_g = Wt*K_o*K_v*K_sg*Pd/Fw*(Km*Kb/J) #4854 psi

# Sf_p = St*Yn/(Kt*Kr)/sigma_p 5.62
# Sf_g = St*Yn/(Kt*Kr)/sigma_g 6.82

#S_h = Sc*Zn/(Kt*Kr)/sigma_contact

#Sc_p = 106400 # fig 4-5 contact endurance strength 322Hr + 29100 psi
#Sc_g = 93500 psi
#Zn given Zn_p = 0.948 Zn_g = 1.4458(10**8/(52/17)**-0.023= 0.973)

#sigma_contact_p = Cp(Wt*K_o*K_v*K_s*Km/dp*10*Cf/I) = 70360
#sigma_contact_g = Cp(Wt*K_o*K_v*K_s*Km/dp*10*Cf/I) = 70660
#Cp = 2300 p.749 Cf =1 I=cos(theta)*sin(theta)/2*Mn*(Mg/Mg+1)=0.121 Mn=1 Mg=52/17 (Gear ratio)

#Shp= 1.69
#Shg = 1.52

#Sfp = 5.62
#Sfg = 6.82

#Pinion
#Sfp = 5.73 Shp**2=2.86

#Gear
#Sfg = 6.96
#Shg**2 = 2.36

Ynp: 0.977 Yng: 0.996
Wt: 164.772
Ksp: 1.043 Ksg: 1.052
Km: 1.387
